# Обработка собранных данных

## Импорт библиотек

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

## Открытие данных

In [49]:
df = pd.read_csv('../data.csv', ';')

In [50]:
df.sample(3)

,physical address,number of rooms,area of apartment,number of floors,apartment floor,price,link,repair,bathroom,view from the windows,terrace,year of construction,elevator,extra,type of house,parking
1308,"ул. Карпинского, д. 50| р-н Индустриальный",2,58.8 м²,22,19.0,6221400.0,https://www.avito.ru/perm/kvartiry/2-k._kvarti...,NaN,NaN,NaN,лоджия,NaN,NaN,NaN,монолитный,NaN
272,"Пермский край, Пермь, ул. Чернышевского, 15Г| ...",2,56.3 м²,19,8.0,4990000.0,https://www.avito.ru/perm/kvartiry/2-k._kvarti...,косметический,раздельный,"на улицу, во двор",балкон,2013,2.0,консьерж,монолитный,"за шлагбаумом во дворе, открытая во дворе"
4159,"Пермский край, Пермь, ул. Солдатова, 38| р-н С...",3,56 м²,14,7.0,4000000.0,https://www.avito.ru/perm/kvartiry/3-k._kvarti...,косметический,раздельный,"на улицу, во двор",лоджия,NaN,1.0,газоснабжение,кирпичный,открытая во дворе


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5564 entries, 0 to 5563
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   physical address       5546 non-null   object 
 1   number of rooms        5551 non-null   object 
 2   area of apartment      5551 non-null   object 
 3   number of floors       5551 non-null   object 
 4   apartment floor        5551 non-null   float64
 5   price                  5551 non-null   float64
 6   link                   5564 non-null   object 
 7   repair                 2662 non-null   object 
 8   bathroom               4215 non-null   object 
 9   view from the windows  3962 non-null   object 
 10  terrace                5340 non-null   object 
 11  year of construction   2527 non-null   object 
 12  elevator               3508 non-null   float64
 13  extra                  2326 non-null   object 
 14  type of house          5551 non-null   object 
 15  park

## Удаление записей с пропусками важных признаков

In [52]:
important_features = [
    "physical address", "number of rooms", "area of apartment", "number of floors",
    "apartment floor", "price", "repair", "bathroom", "terrace", "year of construction",
    "type of house", 
]

In [53]:
new_df = df.copy()
for feature in important_features:
    new_df = new_df[new_df[feature].isna() == False]

In [54]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1939 entries, 0 to 5563
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   physical address       1939 non-null   object 
 1   number of rooms        1939 non-null   object 
 2   area of apartment      1939 non-null   object 
 3   number of floors       1939 non-null   object 
 4   apartment floor        1939 non-null   float64
 5   price                  1939 non-null   float64
 6   link                   1939 non-null   object 
 7   repair                 1939 non-null   object 
 8   bathroom               1939 non-null   object 
 9   view from the windows  1762 non-null   object 
 10  terrace                1939 non-null   object 
 11  year of construction   1939 non-null   object 
 12  elevator               1555 non-null   float64
 13  extra                  1380 non-null   object 
 14  type of house          1939 non-null   object 
 15  park

## Выделение новых признаков из физического адреса

### Получение района города, в котором находится квартира

In [55]:
def get_district(address):
    result = re.search('р-н [а-яА-Я]+', address)
    if result is None:
        return None
    else:
        return result[0][4:]

In [56]:
new_df['district'] = np.array([get_district(item) for item in  new_df['physical address'].values])

In [57]:
print("Найденные районы: " + str(new_df['district'].unique()))

Найденные районы: ['Индустриальный' 'Свердловский' 'Мотовилихинский' 'Кировский'
 'Орджоникидзевский' 'Дзержинский' None 'Ленинский']


In [58]:
print("Примеры нераспознанных адресов: \n" + '\n'.join(new_df[new_df['district'].isna() == True]['physical address'][:10].values))

Примеры нераспознанных адресов: 
Пермский край, Пермский р-н, с. Фролы, Весенняя ул., 18
Пермский край, Пермский р-н, с. Фролы, Весенняя ул., 30
Пермский край, Пермский р-н, с. Фролы, Весенняя ул., 30
Пермский край, Пермский р-н, с. Фролы, Весенняя ул., 32
Пермский край, Пермский р-н, с. Фролы, Весенняя ул., 14А
Пермский край, Пермский р-н, Савинское сельское поселение, д. Песьянка, ул. Строителей, 8
Пермский край, Пермский р-н, Заболотское сельское поселение, д. Горшки, Садовая ул., 1
Пермский край, Пермский р-н, с. Фролы, Центральная ул., 15
Пермский край, Пермский р-н, Култаевское сельское поселение, д. Усть-Тары, ул. Нефтяников, 3
Пермский край, Пермский р-н, с. Фролы, Весенняя ул., 34


Пермский район нас не интересует, поэтому удаляем все эти записи из таблицы

In [59]:
new_df = new_df[new_df['district'].isna() == False]

In [60]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1894 entries, 0 to 5563
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   physical address       1894 non-null   object 
 1   number of rooms        1894 non-null   object 
 2   area of apartment      1894 non-null   object 
 3   number of floors       1894 non-null   object 
 4   apartment floor        1894 non-null   float64
 5   price                  1894 non-null   float64
 6   link                   1894 non-null   object 
 7   repair                 1894 non-null   object 
 8   bathroom               1894 non-null   object 
 9   view from the windows  1720 non-null   object 
 10  terrace                1894 non-null   object 
 11  year of construction   1894 non-null   object 
 12  elevator               1521 non-null   float64
 13  extra                  1363 non-null   object 
 14  type of house          1894 non-null   object 
 15  park

### Получение полного адреса без района

In [61]:
def get_full_address(text):
    address = text.split('|')[0]
    replaces = {
        'ш.': 'шоссе',
        'пр-т': 'проспект',
        'б-р': 'бульвар',
        'пр.': 'проезд'
    }
    for key in replaces:
        if key in address:
            address = address.replace(key, replaces[key])
    return address

In [62]:
new_df['full address'] = np.array([get_full_address(item) for item in new_df['physical address'].values])

In [63]:
print(new_df['full address'].unique())

['Пермский край, Пермь, ул. Архитектора Свиязева, 50/2'
 'Пермский край, Пермь, Кузбасская ул., 41'
 'Пермский край, Пермь, шоссе Космонавтов, 215' ...
 'Пермский край, Пермь, ул. Крисанова, 73'
 'Пермский край, Пермь, посёлок Новые Ляды, Чусовская ул., 24А'
 'Пермский край, Пермь, ул. Гашкова, 12']


In [64]:
new_df = new_df[new_df['full address'].isna() == False]

In [65]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1894 entries, 0 to 5563
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   physical address       1894 non-null   object 
 1   number of rooms        1894 non-null   object 
 2   area of apartment      1894 non-null   object 
 3   number of floors       1894 non-null   object 
 4   apartment floor        1894 non-null   float64
 5   price                  1894 non-null   float64
 6   link                   1894 non-null   object 
 7   repair                 1894 non-null   object 
 8   bathroom               1894 non-null   object 
 9   view from the windows  1720 non-null   object 
 10  terrace                1894 non-null   object 
 11  year of construction   1894 non-null   object 
 12  elevator               1521 non-null   float64
 13  extra                  1363 non-null   object 
 14  type of house          1894 non-null   object 
 15  park

В дальнейшем полный адрес будет использоваться для генерации признаков, связанных с географическим положением квартиры

## Корректировка имеющихся признаков

## Количество комнат

In [66]:
print("Уникальные значения числа комнат: " + ', '.join(new_df['number of rooms'].unique()))

Уникальные значения числа комнат: 3, 1, 2, 6, студия, 4, своб. планировка, 5


In [67]:
def get_number(string):
    if string == 'студия' or string == 'своб. планировка':
        return 1
    else:
        return int(string)

In [68]:
new_df['number of rooms'] = np.array([get_number(item) for item in new_df['number of rooms'].values])

In [69]:
print("Уникальные значения числа комнат после корректировки: " + str(new_df['number of rooms'].unique()))

Уникальные значения числа комнат после корректировки: [3 1 2 6 4 5]


### Площадь квартиры

In [70]:
print("Типичные значения для площади: " + ', '.join(new_df['area of apartment'][:10]))

Типичные значения для площади: 61.2 м², 31.1 м², 89.1 м², 34 м², 30.3 м², 84.3 м², 64.7 м², 37 м², 59.8 м², 31 м²


In [71]:
new_df['area of apartment'] = np.array([float(item[:-3]) for item in new_df['area of apartment'].values])

In [72]:
print("Значения для площади после корректировки: " + ', '.join(list(map(str, new_df['area of apartment'].values[:10]))))

Значения для площади после корректировки: 61.2, 31.1, 89.1, 34.0, 30.3, 84.3, 64.7, 37.0, 59.8, 31.0


### Количество этажей в доме

In [73]:
print("Уникальные значения для числа этажей в доме: \n" + ', '.join(new_df['number of floors'].unique()))

Уникальные значения для числа этажей в доме: 
5, 2, 25, 7, 6, 10, 3, 20, 9, 17, 4, 16, 24, 22, 5 эт., 19, 12, 11, 23, 14, 8, 18, 9 эт., 27, 13, 15, 26, 31, 21, 16 эт., 10 эт., 1


In [74]:
new_df['number of floors'] = np.array([int(re.findall('[\d]+', x)[0]) for x in new_df['number of floors'].values])

In [75]:
print("Уникальные значения для числа этажей в доме после корректировки: \n" + ', '.join(list(map(str, new_df['number of floors'].unique()))))

Уникальные значения для числа этажей в доме после корректировки: 
5, 2, 25, 7, 6, 10, 3, 20, 9, 17, 4, 16, 24, 22, 19, 12, 11, 23, 14, 8, 18, 27, 13, 15, 26, 31, 21, 1


### Этаж квартиры

In [76]:
new_df['apartment floor'].unique()

array([ 1.,  3.,  4.,  5.,  6.,  7.,  2., 10.,  9.,  8., 22., 23., 14.,
       16., 11., 12., 13., 18., 19., 15., 26., 21., 20., 24., 17., 25.])

In [77]:
new_df['apartment floor'] = new_df['apartment floor'].astype(int)

In [78]:
new_df['apartment floor'].unique()

array([ 1,  3,  4,  5,  6,  7,  2, 10,  9,  8, 22, 23, 14, 16, 11, 12, 13,
       18, 19, 15, 26, 21, 20, 24, 17, 25])

### Цена

In [79]:
new_df['price'].dtype

dtype('float64')

Производить манипуляции над ценой не нужно

### Ремонт

In [80]:
print("Уникальные значения для ремонта: " + ', '.join(new_df['repair'].unique()))

Уникальные значения для ремонта: косметический, евро, требует ремонта, дизайнерский


Предобработка не требуется, в дальнейшем данный признак будет закодирован методом one hot encoding

### Санузел

In [81]:
print("Уникальные значения для санузла: " + ', '.join(new_df['bathroom'].unique()))

Уникальные значения для санузла: раздельный, совмещенный, несколько


Предобработка не требуется, в дальнейшем данный признак будет закодирован методом one hot encoding

### Вид из окна

In [82]:
print("Уникальные значения для вида из окна: " + str(new_df['view from the windows'].unique()))

Уникальные значения для вида из окна: ['на улицу, во двор' 'на улицу' 'во двор' nan]


Признак малоинформативен, его нужно будет удалить

### Вид балкона

In [83]:
print("Уникальные значения для санузла: " + ', '.join(new_df['terrace'].unique()))

Уникальные значения для санузла: балкон, нет, лоджия


Предобработка не требуется, в дальнейшем данный признак будет закодирован методом one hot encoding

### Год постройки

In [84]:
print("Уникальные значения для года постройки: " + ', '.join(new_df['year of construction'].unique()))

Уникальные значения для года постройки: 1972, 1957, 2010, 2014, 1966, 2016, 1993, 1960, 1971, 2017, 1978, 1962, 1995, 2015, 2012, 1969, 1996, 1987, 1975, 1953, 1980, 1986, 2019, 2009, 2007, 21, 2018, 1970, 1961, 1964, 1997, 1990, 1965, 1974, 1984, 1959, 1983, 1967, 1954, 2005, 2013, 2011, 2003, 1991, 2001, 2020, 1950, 1979, 1963, 1981, 2021, 1968, 1992, 1985, 1989, 2008, 1973, 2006, 1976, 1998, 2000, 2004, 1982, 1977, 1951, 1988, 1999, 1958, 1929, 2023, 1994, 1956, 1949, 1942, 2002, 1917, 1955, 1946, 63, 84, 78, 1681, 1939, 198, 1940


In [85]:
def year(y):
    y = int(y)
    if y < 100:
        return 1900 + y
    elif y < 1000:
        return y*10
    else:
        return y

In [86]:
new_df['year of construction'] = np.array([year(item) for item in new_df['year of construction'].values])

In [87]:
print("Уникальные значения для года постройки после коррекции: " + str(new_df['year of construction'].unique()))

Уникальные значения для года постройки после коррекции: [1972 1957 2010 2014 1966 2016 1993 1960 1971 2017 1978 1962 1995 2015
 2012 1969 1996 1987 1975 1953 1980 1986 2019 2009 2007 1921 2018 1970
 1961 1964 1997 1990 1965 1974 1984 1959 1983 1967 1954 2005 2013 2011
 2003 1991 2001 2020 1950 1979 1963 1981 2021 1968 1992 1985 1989 2008
 1973 2006 1976 1998 2000 2004 1982 1977 1951 1988 1999 1958 1929 2023
 1994 1956 1949 1942 2002 1917 1955 1946 1681 1939 1940]


### Количество лифтов

In [88]:
print("Уникальные значения для числа лифтов: " + str(new_df['elevator'].unique()))

Уникальные значения для числа лифтов: [ 0.  2.  1. nan  3.  4.]


In [89]:
print("Количество записей с неизвестным числом лифтов: " + str(len(new_df[new_df['elevator'].isna() == True])))

Количество записей с неизвестным числом лифтов: 373


Для заполнения пропусков воспользуемся будем использовать следующее правило:
1. До 5 этажей в ключительно - нет лифтов
2. От 6 до 9 этажей включительно - 1 лифт
3. От 10 до 19 этажей включительно - 2 лифта
4. От 20 этажей - 3 лифта

In [90]:
def get_n_elevator(floor):
    if floor <= 5:
        return 0
    elif 6 <= floor <= 9:
        return 1
    elif 10 <= floor <= 19:
        return 2
    else:
        return 3

In [91]:
new_df.loc[new_df.elevator.isna(), 'elevator'] = np.array([get_n_elevator(item) for item in new_df[new_df.elevator.isna()]['number of floors'].values])

In [92]:
print("Количество записей с неизвестным числом лифтов: " + str(len(new_df[new_df['elevator'].isna() == True])))

Количество записей с неизвестным числом лифтов: 0


In [93]:
print("Уникальные значения для числа лифтов после коррекции: " + str(new_df['elevator'].unique()))

Уникальные значения для числа лифтов после коррекции: [0. 2. 1. 3. 4.]


### Дополнительные услуги

In [94]:
print("Уникальные значения для дополнительных услуг: " + str(new_df['extra'].unique()))

Уникальные значения для дополнительных услуг: ['газоснабжение' 'консьерж' nan 'консьерж, газоснабжение'
 'мусоропровод, газоснабжение' 'консьерж, мусоропровод, газоснабжение'
 'мусоропровод' 'консьерж, мусоропровод']


Из этих признаков нас интересует только мусоропровод, консьерж. Если информации нет, то и признаки эти будут отсутствовать у дома.

In [95]:
def get_garbage(text):
    if 'мусоропровод' in text:
        return 1
    else:
        return 0

def get_concierge(text):
    if 'консьерж' in text:
        return 1
    else:
        return 0

In [96]:
new_df['concierge'] = 0
new_df['garbage chute'] = 0

In [97]:
new_df.loc[new_df.extra.notnull(), 'concierge'] = np.array([get_concierge(item) for item in new_df[new_df.extra.notnull()]['extra'].values])
new_df.loc[new_df.extra.notnull(), 'garbage chute'] = np.array([get_garbage(item) for item in new_df[new_df.extra.notnull()]['extra'].values])

### Тип дома

In [98]:
print("Уникальные значения для типа дома: \n" + '\n'.join(new_df['type of house'].unique()))

Уникальные значения для типа дома: 
панельный
кирпичный
монолитный
блочный
Ленинградский проект . Дом кирпичный. Дома этой серии испытаны временем, проживание в них комфортно и удобно, а сама квартира теплая, сухая, солнечная. Квартира требует ремонта. Есть возможность воплотить все свои дизайнерские фантазии. С/у совмещен , трубы поменяны. Кухня, прихожая - натяжные потолки. По документам - 1 этаж, но со стороны улицы уровень второго этажа!!! Балкон с решетками. Отличный вариант для семьи с детьми, чтобы жить современно и независимо. Дружелюбный дом с большой придомовой территорией и вместительной парковкой. Детская площадка огорожена. Опрятный, чистый подъезд. Разнообразные магазины, торговые центры, д/с, поликлиника, парк с детскими аттракционами - все в шаговой доступности! Отличная транспортная развязка. При покупке квартиры по ипотеке Сбербанка предоставляется скидка от базовой ставки 0.3%. Не упустите свой шанс выгодно приобрести квартиру!
деревянный


Замечен тип дома, принимающий экстремальное значение, это нужно исправить

In [99]:
def get_house_type(hous_type):
    if len(hous_type) < 20:
        return hous_type
    else:
        return "кирпичный"

In [100]:
new_df['type of house'] = np.array([get_house_type(item) for item in new_df['type of house'].values])

In [101]:
print("Уникальные значения для типа дома после коррекции: \n" + '\n'.join(new_df['type of house'].unique()))

Уникальные значения для типа дома после коррекции: 
панельный
кирпичный
монолитный
блочный
деревянный


### Парковка

In [102]:
print("Уникальные значения для парковки: \n" + str(new_df['parking'].unique()))

Уникальные значения для парковки: 
['открытая во дворе' nan 'наземная многоуровневая, за шлагбаумом во дворе'
 'за шлагбаумом во дворе' 'подземная'
 'наземная многоуровневая, открытая во дворе'
 'подземная, наземная многоуровневая, за шлагбаумом во дворе'
 'наземная многоуровневая' 'подземная, за шлагбаумом во дворе'
 'подземная, за шлагбаумом во дворе, открытая во дворе'
 'подземная, наземная многоуровневая, открытая во дворе'
 'наземная многоуровневая, за шлагбаумом во дворе, открытая во дворе'
 'за шлагбаумом во дворе, открытая во дворе'
 'подземная, наземная многоуровневая, за шлагбаумом во дворе, открытая во дворе'
 'подземная, открытая во дворе' 'подземная, наземная многоуровневая']


In [103]:
print("Количество записей с неизвестным типом парковки: " + str(new_df['parking'].isna().sum()))

Количество записей с неизвестным типом парковки: 254


In [104]:
new_df.loc[new_df.parking.isna(), 'parking'] = ""

Этот признак, несмотря на большое количество пропусков, можно оставить, так как этот признак будет разбит на несколько категорий, исходя из типа парковки, методом one hot encoding.

## Удаление ненужных столбцов

In [105]:
new_df.columns

Index(['physical address', 'number of rooms', 'area of apartment',
       'number of floors', 'apartment floor', 'price', 'link', 'repair',
       'bathroom', 'view from the windows', 'terrace', 'year of construction',
       'elevator', 'extra', 'type of house', 'parking', 'district',
       'full address', 'concierge', 'garbage chute'],
      dtype='object')

In [106]:
column_for_delete = ["physical address", "link", "view from the windows", "extra"]

In [107]:
for column in column_for_delete:
    new_df = new_df.drop(column, axis=1)

In [108]:
new_df.columns

Index(['number of rooms', 'area of apartment', 'number of floors',
       'apartment floor', 'price', 'repair', 'bathroom', 'terrace',
       'year of construction', 'elevator', 'type of house', 'parking',
       'district', 'full address', 'concierge', 'garbage chute'],
      dtype='object')

In [109]:
new_df.index = np.arange(len(new_df))

## Итог первичной обработки

In [110]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1894 entries, 0 to 1893
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   number of rooms       1894 non-null   int32  
 1   area of apartment     1894 non-null   float64
 2   number of floors      1894 non-null   int32  
 3   apartment floor       1894 non-null   int32  
 4   price                 1894 non-null   float64
 5   repair                1894 non-null   object 
 6   bathroom              1894 non-null   object 
 7   terrace               1894 non-null   object 
 8   year of construction  1894 non-null   int32  
 9   elevator              1894 non-null   float64
 10  type of house         1894 non-null   object 
 11  parking               1894 non-null   object 
 12  district              1894 non-null   object 
 13  full address          1894 non-null   object 
 14  concierge             1894 non-null   int64  
 15  garbage chute        

In [111]:
new_df.to_csv('../clean_data.csv', sep=';', index=False)

__В ходе первой обработки были:__
1. удалены записи с пропусками важных признаков
2. выделены признаки района и полного адреса без района
3. рассмотрены и скорректированы некоторые признаки
4. удалены ненужные столбцы